In [10]:
%%time

import cv2
import shutil
import numpy as np
import os
# Directory paths
input_dir = "/Users/lucas.wojahn/Documents/GitHub/duke-dbt-data/data/resized/"
output_dir = "/Users/lucas.wojahn/Documents/GitHub/duke-dbt-data/data/augmented/"

# Create output directory if not exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define parameters for Gaussian Noise
mean = 0
stddev = 0.1

# Define parameters for Color Channel Shifting
shift_range = 0.2

# Define parameters for Color Transformation
brightness_range = (0, 50)
contrast_range = (0.5, 1.5)

# Function to apply Gaussian Noise
def apply_gaussian_noise(image):
    gaussian_noise = np.random.normal(mean, stddev, image.shape)
    noisy_image = image + gaussian_noise
    return np.clip(noisy_image, 0, 1)

# Function to apply Color Channel Shifting
def apply_color_channel_shifting(image):
    r_shift = np.random.uniform(-shift_range, shift_range)
    g_shift = np.random.uniform(-shift_range, shift_range)
    b_shift = np.random.uniform(-shift_range, shift_range)
    shifted_image = np.clip(image + [r_shift, g_shift, b_shift], 0, 1)
    return shifted_image

# Function to apply Color Transformation
def apply_color_transformation(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv[..., 0] += np.random.randint(*brightness_range)
    hsv[..., 1] *= np.random.uniform(*contrast_range)
    hsv[..., 2] *= np.random.uniform(*contrast_range)
    transformed_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return transformed_image

def generate_annotation(filename):
    txt_file = filename.replace('.jpg','.txt')
    original_annotation = os.path.join(input_dir,txt_file)

    shutil.copy(original_annotation, os.path.join(output_dir, "noisy_" + txt_file))
    shutil.copy(original_annotation, os.path.join(output_dir, "shifted_" + txt_file))
    shutil.copy(original_annotation, os.path.join(output_dir, "transformed_" + txt_file))


# Loop through all image files in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Load the image
        image = cv2.imread(os.path.join(input_dir, filename))

        # Convert the image to float32 for data augmentations
        image = image.astype(np.float32) / 255.0

        # Apply Gaussian Noise
        noisy_image = apply_gaussian_noise(image)

        # Apply Color Channel Shifting
        shifted_image = apply_color_channel_shifting(image)

        # Apply Color Transformation
        transformed_image = apply_color_transformation(image)

        # Convert the images back to uint8
        noisy_image = (noisy_image * 255).astype(np.uint8)
        shifted_image = (shifted_image * 255).astype(np.uint8)
        transformed_image = (transformed_image * 255).astype(np.uint8)

        # Save the augmented images to the output directory
        noisy_filename = os.path.join(output_dir, "noisy_" + filename)
        shifted_filename = os.path.join(output_dir, "shifted_" + filename)
        transformed_filename = os.path.join(output_dir, "transformed_" + filename)

        cv2.imwrite(noisy_filename, noisy_image)
        cv2.imwrite(shifted_filename, shifted_image)
        cv2.imwrite(transformed_filename, transformed_image)

        generate_annotation(filename)


CPU times: user 6.31 s, sys: 912 ms, total: 7.23 s
Wall time: 7.19 s
